<a href="https://colab.research.google.com/github/smpark0520/ESAA/blob/main/5_23_%EC%84%B8%EC%85%98_%ED%95%A9%EC%84%B1%EA%B3%B1_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. CIFAR-10 데이터셋을 이용한 CNN 모델

## Step 1. 필요한 라이브러리 임포트

In [1]:
# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [2]:
# 디바이스 설정 (GPU 사용 가능하면 GPU, 없으면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Step 2. CIFAR-10 데이터셋 다운로드

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 정규화 추가
])

# 데이터셋 로드
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)


100%|██████████| 170M/170M [00:06<00:00, 25.2MB/s]


##Step 3. DataLoader 설정

In [4]:
# DataLoader를 사용하여 배치 크기 64로 데이터를 로드하세요.
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

## Step 4. CNN 모델 정의

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class CIFAR10_CNN(nn.Module):
    def __init__(self):
        super(CIFAR10_CNN, self).__init__()

        # 첫 번째 합성곱 층: 입력 채널 3, 출력 채널 32
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        # 두 번째 합성곱 층: 입력 채널 32, 출력 채널 64
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        # 세 번째 합성곱 층: 입력 채널 64, 출력 채널 128
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        # 풀링 층: 2x2 크기, stride=2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully Connected Layer
        self.fc1 = nn.Linear(128 * 4 * 4, 256)  # 32x32 → 16x16 → 8x8 → 4x4 (3번 maxpool)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)  # CIFAR-10은 클래스 10개

        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))

        x = x.view(x.size(0), -1)  # Flatten

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Step 5. 모델 학습 설정

In [7]:
import torch.optim as optim

# 모델을 초기화하세요.
model = CIFAR10_CNN()


# 손실 함수로 CrossEntropyLoss를 사용하고 옵티마이저는 Adam을 사용하세요.
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Step 6. 모델 학습 루프

In [8]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()  # 학습 모드 설정
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 모델을 통해 예측값 계산
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 경사 초기화
        optimizer.zero_grad()

        # 역전파
        loss.backward()

        # 최적화 (가중치 갱신)
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/3], Loss: 1.2929
Epoch [2/3], Loss: 0.9178
Epoch [3/3], Loss: 0.7761


## Step 7. 테스트 정확도 평가

In [9]:
# 모델의 성능 평가

correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 71.10%


# 2. MNIST 데이터셋 분류 모델




## Step 1. 필요한 라이브러리 임포트

In [10]:
# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [11]:
# 디바이스 설정 (GPU 사용 가능하면 GPU, 없으면 CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Step 2. MNIST 데이터셋 다운로드

In [12]:
# MNIST 데이터셋을 다운로드하고, 텐서로 변환할 수 있도록 필요한 전처리 함수를 추가하세요.
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)


100%|██████████| 9.91M/9.91M [00:00<00:00, 16.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 495kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.57MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 3.82MB/s]


##Step 3. DataLoader 설정

In [13]:
# DataLoader를 사용하여 배치 크기 64로 데이터를 로드하세요.
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

## Step 4. CNN 모델 정의

In [14]:
# CNN 모델 정의
class MNIST_CNN(nn.Module):
    def __init__(self):
        super(MNIST_CNN, self).__init__()
        # 입력 채널 1 (흑백), 출력 채널 32, 커널 크기 3
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)  # 2x2 맥스 풀링
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)

        # Fully Connected Layers
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # MNIST 28x28 → 14x14 → 7x7 after pooling 2번
        self.fc2 = nn.Linear(128, 10)  # 10개 클래스

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Conv1 + ReLU + Pool
        x = self.pool(F.relu(self.conv2(x)))  # Conv2 + ReLU + Pool
        x = x.view(-1, 64 * 7 * 7)            # Flatten
        x = F.relu(self.fc1(x))                # FC1 + ReLU
        x = self.fc2(x)                        # FC2 (출력)
        return x

## Step 5. 모델 학습 설정

In [15]:
# 모델을 초기화하세요.
model = MNIST_CNN()

# 손실 함수로 CrossEntropyLoss를 사용하고 옵티마이저는 Adam을 사용하세요.
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Step 6. 모델 학습 루프

In [16]:
# 모델 학습을 위한 루프를 작성하세요. 에폭 수는 3로 설정합니다.
# 런타임 약 5분

num_epochs = 3

for epoch in range(num_epochs):
    model.train()  # 학습 모드
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()           # 경사 초기화
        outputs = model(images)         # 순전파
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()                 # 역전파
        optimizer.step()                # 최적화

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/3], Loss: 0.1794
Epoch [2/3], Loss: 0.0492
Epoch [3/3], Loss: 0.0339


## Step 7. 테스트 정확도 평가

In [17]:
# 모델의 성능 평가
correct = 0
total = 0
model.eval()  # 평가 모드 설정
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # 가장 높은 값의 인덱스

        total += labels.size(0)  # 전체 샘플 수 누적
        correct += (predicted == labels).sum().item()  # 맞은 개수 누적

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 99.12%
